Fullname: Lê Hữu Sang

Student ID: 22120310

# HW3: CUDA Memory

**To compile your file, you can use this command:** \
`nvcc filename.cu -o execute_filename` \
***You can use Vietnamese to anwser the questions***

You can add/delete cells as you see fit. Just don't delete the cells with the Teacher's words.

---

Below is the command to install `Nsight system` on Colab. You just need to run the cell.

In [1]:
# Here is the command to install Nsight system on Colab. You just need to run the cell.
!apt-get update
!apt-get install -y cuda-nsight-systems-12-2

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

- The above installation will be remove when colab restart its runtime. It should be fine for this homework because we don't use Nsight system that much.
- But if you want to persistent install Nsight system, you mount space from your Google Drive to your Colab VM. Read [Here](https://stackoverflow.com/questions/76784746/how-to-use-nsys-in-google-colab) for guidelines how to do thhis.

In [2]:
from numba import cuda

major, minor = cuda.get_current_device().compute_capability
print(f"GPU compute capability: {major}.{minor}")

GPU compute capability: 7.5


In [3]:
from PIL import Image
import matplotlib.pyplot as plt

def show_pnm_side_by_side(file_list, titles=None, figsize=(15, 5)):
    n = len(file_list)
    plt.figure(figsize=figsize)

    for i, path in enumerate(file_list):
        try:
            img = Image.open(path)
        except Exception as e:
            print(f"Cannot open {path}: {e}")
            continue

        plt.subplot(1, n, i + 1)
        cmap = "gray" if img.mode == "L" else None
        plt.imshow(img, cmap=cmap)
        title = titles[i] if titles and i < len(titles) else path
        plt.title(title)
        plt.axis("off")

    plt.tight_layout()
    plt.show()


---

## Exercises 1

In [4]:
!nvcc -arch=sm_{major}{minor} HW3_P1.cu -o HW3_P1

In [5]:
!./HW3_P1 in.pnm out.pnm

**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15828320256 bytes
CMEM: 65536 bytes
L2 cache: 4194304 bytes
SMEM / one SM: 65536 bytes
****************************

Image size (width x height): 512 x 512

Kernel 1, block size 16x16, grid size 32x32
Kernel time: 0.716832 ms
Error: 0.000703

Kernel 2, block size 16x16, grid size 32x32
Kernel time: 0.342016 ms
Error: 0.000703

Kernel 3, block size 16x16, grid size 32x32
Kernel time: 0.278496 ms
Error: 0.000703


## Nhận xét chi tiết

### 1) Kernel 1 – đọc dữ liệu thô từ Global Memory
Phiên bản này mỗi thread phải tự lấy toàn bộ 81 giá trị từ global memory để thực hiện convolution.  
Dù GPU có cache hỗ trợ, lượng truy cập vẫn rất lớn, và **bandwidth DRAM trở thành giới hạn chính**.  
**Ưu điểm:** dễ cài đặt.

---

### 2) Kernel 2 – dùng Shared Memory để chứa vùng tile
Ở phiên bản này, cả block hợp tác nạp toàn bộ ô dữ liệu đầu vào (bao gồm cả biên) lên shared memory **một lần duy nhất**.  
Phần dữ liệu này sau đó được **tái sử dụng cho tất cả 256 pixels output** trong cùng block.

**Ưu điểm:**

- Số lần đọc từ DRAM giảm xuống chỉ còn một phần nhỏ so với kernel 1  
- Truy cập shared memory nhanh hơn nhiều vì độ trễ thấp và nằm ngay trong SM  
- Chi phí đồng bộ (`__syncthreads()`) chỉ một vài lần và nhỏ hơn rất nhiều so với lượng dữ liệu tiết kiệm được  

**Kết quả thực tế:** thời gian giảm gần **2×**.

---

### 3) Kernel 3 – combination của Shared Memory + Constant Memory
Phiên bản cuối vừa tận dụng tile trong shared memory, vừa đưa toàn bộ **filter** vào constant memory.

**Ưu điểm:**

- Bộ lọc kích thước nhỏ, phù hợp với constant memory  
- Mỗi bước tính, toàn bộ thread trong warp đọc cùng một hệ số  
- GPU có cơ chế **broadcast** cho cả warp chỉ trong một giao dịch  
- Hiệu quả cache gần như tối đa (hit rate rất cao)

**Kết quả:** phần đọc hệ số filter từ global memory gần như bị loại bỏ hoàn toàn, thời gian tiếp tục giảm thêm khoảng **20–25%** so với kernel 2.


## Exercises 2

In [6]:
!nvcc -arch=sm_{major}{minor} HW3_P2.cu -o HW3_P2

In [7]:
!./HW3_P2

╔════════════════════════════════════════╗
║  VECTOR OPERATIONS PIPELINE EXERCISE   ║
╚════════════════════════════════════════╝

Configuration:
  Number of vectors: 8
  Elements per vector: 4194304 (16.00 MB)
  Total data: 128.00 MB

GPU: Tesla T4
Concurrent kernels supported: Yes
Number of copy engines: 3


=== SEQUENTIAL (NO STREAMS) ===
Time: 25.45 ms

=== BREADTH-FIRST PIPELINE ===
Time: 15.51 ms

=== DEPTH-FIRST PIPELINE ===
Time: 15.60 ms

=== VERIFICATION ===
✓ All results match!

╔════════════════════════════════════════╗
║         PERFORMANCE SUMMARY            ║
╚════════════════════════════════════════╝
Sequential:     25.45 ms
Breadth-First:  15.51 ms  (1.64x speedup)
Depth-First:    15.60 ms  (1.63x speedup)

Depth-First vs Breadth-First: 0.99x


In [8]:
!./HW3_P2 16

╔════════════════════════════════════════╗
║  VECTOR OPERATIONS PIPELINE EXERCISE   ║
╚════════════════════════════════════════╝

Configuration:
  Number of vectors: 16
  Elements per vector: 4194304 (16.00 MB)
  Total data: 256.00 MB

GPU: Tesla T4
Concurrent kernels supported: Yes
Number of copy engines: 3


=== SEQUENTIAL (NO STREAMS) ===
Time: 50.04 ms

=== BREADTH-FIRST PIPELINE ===
Time: 32.62 ms

=== DEPTH-FIRST PIPELINE ===
Time: 30.44 ms

=== VERIFICATION ===
✓ All results match!

╔════════════════════════════════════════╗
║         PERFORMANCE SUMMARY            ║
╚════════════════════════════════════════╝
Sequential:     50.04 ms
Breadth-First:  32.62 ms  (1.53x speedup)
Depth-First:    30.44 ms  (1.64x speedup)

Depth-First vs Breadth-First: 1.07x


In [9]:
!./HW3_P2 32

╔════════════════════════════════════════╗
║  VECTOR OPERATIONS PIPELINE EXERCISE   ║
╚════════════════════════════════════════╝

Configuration:
  Number of vectors: 32
  Elements per vector: 4194304 (16.00 MB)
  Total data: 512.00 MB

GPU: Tesla T4
Concurrent kernels supported: Yes
Number of copy engines: 3


=== SEQUENTIAL (NO STREAMS) ===
Time: 100.63 ms

=== BREADTH-FIRST PIPELINE ===
Time: 68.56 ms

=== DEPTH-FIRST PIPELINE ===
Time: 61.83 ms

=== VERIFICATION ===
✓ All results match!

╔════════════════════════════════════════╗
║         PERFORMANCE SUMMARY            ║
╚════════════════════════════════════════╝
Sequential:     100.63 ms
Breadth-First:  68.56 ms  (1.47x speedup)
Depth-First:    61.83 ms  (1.63x speedup)

Depth-First vs Breadth-First: 1.11x


In [10]:
# Generate report with nsight system
!nsys profile ./HW3_P2

╔════════════════════════════════════════╗
║  VECTOR OPERATIONS PIPELINE EXERCISE   ║
╚════════════════════════════════════════╝

Configuration:
  Number of vectors: 8
  Elements per vector: 4194304 (16.00 MB)
  Total data: 128.00 MB

GPU: Tesla T4
Concurrent kernels supported: Yes
Number of copy engines: 3


=== SEQUENTIAL (NO STREAMS) ===
Time: 25.64 ms

=== BREADTH-FIRST PIPELINE ===
Time: 15.54 ms

=== DEPTH-FIRST PIPELINE ===
Time: 15.48 ms

=== VERIFICATION ===
✓ All results match!

╔════════════════════════════════════════╗
║         PERFORMANCE SUMMARY            ║
╚════════════════════════════════════════╝
Sequential:     25.64 ms
Breadth-First:  15.54 ms  (1.65x speedup)
Depth-First:    15.48 ms  (1.66x speedup)

Depth-First vs Breadth-First: 1.00x
Generating '/tmp/nsys-report-9137.qdstrm'
[1/1] [========================100%] report2.nsys-rep
Generated:
    /content/report2.nsys-rep


# Phần B — Phân tích hiệu năng

## Question 1: Performance Analysis Why is depth-first faster than breadth-first?

**Depth-first (DF):**
- Với DF, mỗi vector được xử lý toàn bộ pipeline **(H2D → kernels → D2H)** trên **một stream**.
- Nhờ vậy, giữa các stream có thể **overlap copy–compute** (khi dùng pinned memory và GPU có nhiều copy engines).
- Khi stream A chạy kernel, stream B đang copy dữ liệu → tận dụng tài nguyên tốt hơn.

**Breadth-first (BF):**
- BF thực hiện cùng một pha cho tất cả vector (copy hết H2D, rồi chạy tất cả kernel, rồi copy hết D2H).
- Tạo ra **burst load** lên PCIe và hàng đợi kernel.
- Khó đạt overlap copy–compute vì toàn bộ pipeline bị dồn theo chiều ngang.

**Kết luận:**
- DF thường nhanh hơn BF nhờ tránh nghẽn tài nguyên, phân bố đều công việc và tạo nhiều cơ hội overlap copy–compute.


## Question 2: Scalability
+ What happens if you increase the number of vectors to 16? 32?
+ Students should experiment and plot results

### **1. Kết quả với 16 vectors**

| Phương pháp      | Thời gian (ms) | Speedup so với Sequential |
|------------------|----------------|----------------------------|
| Sequential       | 51.06 ms       | 1.00×                      |
| Breadth-First    | 33.23 ms       | 1.54×                      |
| Depth-First      | 31.07 ms       | 1.64×                      |

**Nhận xét:**  
Khi tăng lên 16 vectors, cả hai mô hình pipeline đều nhanh hơn rõ rệt so với sequential. Depth-First tiếp tục nhanh nhất vì khả năng overlap giữa H2D, compute và D2H tốt hơn. Breadth-First cũng cải thiện nhưng vẫn chậm hơn DF một chút do copy theo “đợt lớn” dễ làm nghẽn băng thông.

---

### **2. Kết quả với 32 vectors**

| Phương pháp      | Thời gian (ms) | Speedup so với Sequential |
|------------------|----------------|----------------------------|
| Sequential       | 100.11 ms      | 1.00×                      |
| Breadth-First    | 68.56 ms       | 1.46×                      |
| Depth-First      | 61.76 ms       | 1.62×                      |

**Nhận xét:**  
Với 32 vectors, thời gian chạy của sequential tăng gần như đúng tuyến tính.  
Hai pipeline vẫn nhanh hơn nhiều, nhưng hiệu suất bắt đầu giảm vì băng thông PCIe và tài nguyên copy-engine bị chia sẻ cho quá nhiều stream. Dù vậy, Depth-First vẫn giữ được tốc độ tốt nhất, vượt Breadth-First khoảng 1.11×.

---

### **3. Kết luận**

- Khi số vector tăng, tổng thời gian chạy tăng theo, nhưng pipeline giúp che bớt thời gian copy và tăng khả năng overlap.
- **Depth-First mở rộng tốt nhất**, giữ được mức overlap cao giữa copy và tính toán.
- **Breadth-First mở rộng vừa phải**, nhưng kém hiệu quả hơn khi số vectors lớn do tắc nghẽn băng thông.
- **Sequential tăng hoàn toàn tuyến tính**, không có overlap nên chậm nhất.

## Question 3: Memory Transfer
How much time is spent on memory transfers vs computation?
Student can modift the code to get these time duration.
For example: 30-40% transfer, 60-70% compute. Streams help overlap these components